# Korean LLM (Large Language Model) Serving on SageMaker with Hugging Face TGI (Text Generation Inference)
---

한국어 LLM 모델 SageMaker 서빙 핸즈온 (허깅페이스 허브에서 모델을 그대로 배포). TGI (Text Generation Inference) 라이브러리 사용

- [Hugging Face TGI Repository](https://github.com/huggingface/text-generation-inference)
- [Hugging Face Blog: Introducing the Hugging Face LLM Inference Container for Amazon SageMaker](https://huggingface.co/blog/sagemaker-huggingface-llm)
- [AWS Blog: Announcing the launch of new Hugging Face LLM Inference containers on Amazon SageMaker](https://aws.amazon.com/ko/blogs/machine-learning/announcing-the-launch-of-new-hugging-face-llm-inference-containers-on-amazon-sagemaker/)


---
출처: [AWS AIML GenAI workshop for Korean language](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/jumpstart/text_to_text/%5Bmodel_consumer%5Dkullm_polyglot_12_8b_in_context_learning_ml_g5_12xl.ipynb)

이 모델은 SageMaker Deployment에서 [text-generation-inference](https://github.com/huggingface/text-generation-inference/tree/main)를 활용하여 Endpoint를 생성합니다. text-generation-inference 는 텍스트 생성 추론을 위한 Rust, Python 및 gRPC 서버이며, HuggingFace의 production에서 LLM의 API 추론 위젯을 구동하는 데 사용됩니다.

[주요 특성]
- 간단한 launcher로 가장 인기 있는 대규모 언어 모델 제공
- 여러 GPU에서 더 빠른 추론을 위한 텐서 병렬 처리
- 서버 전송 이벤트(SSE, Server-Sent Events)를 사용한 토큰 스트리밍
- 총 처리량 증가를 위한 수신 요청의 지속적인 batching 처리
- 가장 많이 사용되는 아키텍처에 flash-attention을 사용하여 추론하도록 최적화된 transformers 코드
- bitsandbytes을 이용한 Quantization
- Safetensors 가중치 로딩
- 대규모 언어 모델용 워터마크를 사용한 워터마킹
- Logits 와퍼(temperature 스케일링, top-p, top-k, repetition penalty, 자세한 내용은 transformers.LogitsProcessor 참조)
- 시퀀스 중지
- 로그 확률
- 프로덕션 준비 완료(Open Telemetry, Prometheus metrics를 사용한 분산 추적)


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates')

In [ ]:
# !pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed

<br>

## 1. Serve LLM Model on SageMaker

---

### Imports 

In [ ]:
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.model import Model
from sagemaker import script_uris, image_uris, model_uris, get_execution_role
from sagemaker.utils import name_from_base
import logging
import boto3
import time
import json
import pprint

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

### Setup essentials

In [ ]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

logger.info(f'Using sagemaker=={sagemaker.__version__}')
logger.info(f'Using boto3=={boto3.__version__}')

In [ ]:
# MODEL_ID = 'kullm-polyglot-5-8b-v2'  # the error is that "Expected (head_size % 8 == 0) && (head_size <= 128) to be true, but got false. "
MODEL_ID = "nlpai-lab/kullm-polyglot-12.8b-v2"
MODEL_PREFIX = MODEL_ID.split('/')[-1].replace('.', '-')

MODEL_VERSION = '*'
INSTANCE_TYPE = 'ml.g5.24xlarge'
INSTANCE_COUNT = 1
IMAGE_SCOPE = 'inference'
MODEL_DATA_DOWNLOAD_TIMEOUT = 3600  # in seconds
CONTAINER_STARTUP_HEALTH_CHECK_TIMEOUT = 360
EBS_VOLUME_SIZE = 256  # in GB
CONTENT_TYPE = 'application/json'

# set up roles and clients 
client = boto3.client('sagemaker-runtime')
ROLE = get_execution_role()
logger.info(f'Role => {ROLE}')

### Retrieve Hugging Face LLM DLC for TGI

- https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-text-generation-inference-containers

In [ ]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
deploy_image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)
deploy_image_uri

In [ ]:
env = {
    'HF_TASK': 'text-generation',
    'HF_MODEL_ID': MODEL_ID,
    'SAGEMAKER_MODEL_SERVER_TIMEOUT': str(3600),
    'SM_NUM_GPUS': '4',  ## ml.g5.12xlarge 기준
    'HF_MODEL_QUANTIZE': 'bitsandbytes',  ##[possible values: bitsandbytes, gptq]
}

In [ ]:
endpoint_name =  name_from_base(f"{MODEL_PREFIX}-tgi")
logger.info(f'Endpoint name: {endpoint_name}')

model = HuggingFaceModel(
    image_uri=deploy_image_uri,
    env=env,
    role=ROLE,
    name=endpoint_name
)

### Deploy model to SageMaker Endpoint

In [ ]:
%%time

predictor = model.deploy(
    initial_instance_count=INSTANCE_COUNT, 
    instance_type=INSTANCE_TYPE, 
    endpoint_name=endpoint_name, 
    # volume_size=EBS_VOLUME_SIZE, # If using an instance with local SSD storage, volume_size must be None, e.g. p4 but not p3    
    model_data_download_timeout=MODEL_DATA_DOWNLOAD_TIMEOUT, 
    container_startup_health_check_timeout=CONTAINER_STARTUP_HEALTH_CHECK_TIMEOUT, 
    wait=False
)

In [ ]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
%%time 
from inference_lib import describe_endpoint, Prompter
describe_endpoint(endpoint_name)

<br>

## 2. Inference
---

엔드포인트를 호출할 때 이 텍스트를 JSON 페이로드 내에 제공해야 합니다. 이 JSON 페이로드에는 length, sampling strategy, output token sequence restrictions을 제어하는 데 도움이 되는 원하는 추론 매개변수가 포함될 수 있습니다. 허깅페이스 트랜스포머 transformers 라이브러리에는 [사용 가능한 페이로드 매개변수](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig)의 전체 목록이 정의되어 있지만, 중요한 페이로드 매개변수는 다음과 같이 정의되어 있습니다:

* **do_sample (`bool`)** – logits sampling 활성화
* **max_new_tokens (`int`)** – 생성된 토큰의 최대 수
* **best_of (`int`)** – best_of 개의 시퀀스를 생성하고 가장 높은 토큰 로그 확률이 있는 경우 반환
* **repetition_penalty (`float`)** – 반복 패널티에 대한 파라미터, 1.0은 패널티가 없음을 의미하여 Greedy 서치와 동일, 커질수록 다양한 결과를 얻을 수 있으며, 자세한 사항은 [this paper](https://arxiv.org/pdf/1909.05858.pdf)을 참고
* **return_full_text (`bool`)** – 생성된 텍스트에 프롬프트를 추가할지 여부
* **seed (`int`)** – Random sampling seed
* **stop_sequences (`List[str]`)** – `stop_sequences` 가 생성되면 토큰 생성을 중지
* **temperature (`float`)** – logits 분포 모듈화에 사용되는 값
* **top_k (`int`)** – 상위 K개 만큼 가장 높은 확률 어휘 토큰의 수
* **top_p (`float`)** – 1 보다 작게 설정하게 되며, 상위부터 정렬했을 때 가능한 토큰들의 확률을 합산하여 `top_p` 이상의 가장 작은 집합을 유지
* **truncate (`int`)** – 입력 토큰을 지정된 크기로 잘라냄
* **typical_p (`float`)** – typical Decoding 양으로, 자세한 사항은 [Typical Decoding for Natural Language Generation](https://arxiv.org/abs/2202.00666)을 참고
* **watermark (`bool`)** –  [A Watermark for Large Language Models](https://arxiv.org/abs/2301.10226)가 Watermarking
* **decoder_input_details (`bool`)** – decoder input token logprobs와 ids를 반환

In [ ]:
params = {
    "do_sample": False,
    "max_new_tokens": 512,
    "temperature": 0.2,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.3,
    "presence_penalty": None,
    "eos_token_id": 2
}
    
params = {
    "do_sample": False,
    "max_new_tokens": 128,
    "temperature": 0.7,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}


In [ ]:
input_text = """
고객님: 안녕하세요, iPhone에 문제가 있습니다.
상담원: 안녕하세요! 무슨 문제인가요?
고객님: 휴대폰이 제대로 충전되지 않고 배터리가 매우 빨리 소모되는 것 같습니다. 다른 충전 케이블과 전원 어댑터를 사용해 보았지만 문제가 지속됩니다.
상담원: 흠, 그렇군요. 몇 가지 문제 해결 단계를 시도해 보겠습니다. 설정, 배터리로 이동하여 배터리 수명을 많이 소모하는 앱이 있는지 확인해 주시겠어요?
고객님: 예, 배터리를 많이 소모하는 앱이 몇 개 있습니다.
상담원: 좋아요, 화면 하단에서 위로 스와이프하여 해당 앱을 강제 종료한 다음 앱을 위로 스와이프하여 닫아 보세요.
고객: 그렇게 했는데도 문제가 여전히 남아 있습니다.
상담원: 네, iPhone의 설정을 기본값으로 재설정해 보겠습니다. 이렇게 해도 데이터가 삭제되지는 않습니다. 설정, 일반, 재설정으로 이동한 다음 모든 설정 재설정을 선택하세요.
고객님: 그렇게 했습니다. 다음은 어떻게 해야 하나요?
상담원: 이제 iPhone을 재시동해 보겠습니다. "밀어서 전원 끄기" 옵션이 표시될 때까지 전원 버튼을 길게 누릅니다. 밀어 전원을 끄고 몇 초간 기다린 다음 iPhone을 다시 켜세요.
고객님: 다시 시작했지만 여전히 제대로 충전되지 않습니다.
상담원: 그렇군요. iPhone에서 진단 테스트를 실행해야 할 것 같습니다. 가까운 Apple Store 또는 공인 서비스 제공업체를 방문하여 iPhone을 점검받으시기 바랍니다.
고객: 예약을 해야 하나요?
상담원: 예. 줄을 서서 기다리지 않으려면 항상 미리 예약하는 것이 가장 좋습니다. 온라인으로 예약하거나 Apple Store 또는 공인 서비스 제공업체에 전화하여 예약할 수 있습니다.
고객님: 수리 비용은 제가 지불해야 하나요?
상담원: iPhone에 보증이 적용되는지 여부에 따라 다릅니다. 보증이 적용되는 경우에는 비용을 지불할 필요가 없습니다. 그러나 보증이 적용되지 않는 경우에는 수리 비용을 지불하셔야 합니다.
고객님: iPhone을 돌려받는 데 얼마나 걸리나요?
상담원: 문제의 심각도에 따라 다르지만 일반적으로 영업일 기준 1~2일이 소요됩니다.
고객: 온라인으로 수리 상태를 추적할 수 있나요?
상담원: 온라인 또는 Apple Store 또는 공인 서비스 제공업체에 전화하여 수리 상태를 추적할 수 있습니다.
고객: 알겠습니다. 도와주셔서 감사합니다.
"""

Generation configuration 

In [ ]:
import json
from inference_lib import KoLLMSageMakerEndpoint

ep = KoLLMSageMakerEndpoint(endpoint_name)

### A. Text Summarization 

In [ ]:
%%time
instruction = "다음 대화를 요약해 주세요"
payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

In [ ]:
%%time 
from inference_lib import describe_endpoint, Prompter
#describe_endpoint(endpoint_name)         

### B. Abstractive Question Answering 

##### Q1

In [ ]:
%%time
instruction = 'iPhone 충전 문제를 해결하기 위해 고객에게 어떤 문제 해결 단계를 제안하나요?'
payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

Q2

In [ ]:
%%time
instruction = 'iPhone을 기본 설정으로 재설정하면 충전 문제와 배터리 소모 문제를 해결할 수 있나요?'
payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

Q3

In [ ]:
%%time
instruction = '고객이 iPhone 수리를 위해 가까운 Apple Store 또는 공인 서비스 제공업체에 예약하려면 어떤 조치를 취해야 하나요?'
payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

### C. Sentiment Analysis

In [ ]:
%%time
instruction = '고객과 상담원 간의 대화에 대한 전반적인 감정 점수는 얼마인가요?'
payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

In [ ]:
%store endpoint_name

<br>

## 2. Clean Up
---

In [ ]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

<br>

# References
---

- Model 정보
    - kullm-polyglot-5.8b-v2
        - This model is a parameter-efficient fine-tuned version of EleutherAI/polyglot-ko-5.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2        
    - kullm-polyglot-12.8b-v2
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b      
- 코드
    - [Boto3](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/pytorch_deploy_large_GPT_model/GPT-J-6B-model-parallel-inference-DJL.ipynb)
    - [Python SDK](https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb)
    - [Kor LLM on SageMaker](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker)
    - [AWS Generative AI Workshop for Korean language](https://github.com/aws-samples/aws-ai-ml-workshop-kr/tree/master/genai)